In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/enz-eff-project
!pip install -r requirements.txt

/content/drive/.shortcut-targets-by-id/1iS6gSWfUE3cZnmrNWbbV9_W_zQH3vaiu/enz-eff-project
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.9/191.9 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 

In [ ]:
%cd improved_code/model_training

/content/drive/.shortcut-targets-by-id/1iS6gSWfUE3cZnmrNWbbV9_W_zQH3vaiu/enz-eff-project/improved_code/model_training


In [ ]:
import numpy as np
import pickle
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import r2_score, mean_squared_error
from scipy import stats
import xgboost as xgb

## Loading training and test data:

In [ ]:
train_indices = list(np.load(join("..", "..", "data", "kcat_data", "splits", "CV_train_indices.npy"), allow_pickle = True))
test_indices = list(np.load(join("..", "..", "data", "kcat_data", "splits", "CV_test_indices.npy"), allow_pickle = True))

In [ ]:
data_train = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl"))
data_test = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl"))

# print((train_indices))
# print((test_indices))

data_train.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
data_test.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
len(data_train), len(data_test)

(3391, 874)

In [ ]:
data_train.columns

Index(['Reaction ID', 'Sequence ID', 'kcat_values', 'Uniprot IDs',
       'from_BRENDA', 'from_Sabio', 'from_Uniprot', 'checked', 'Sequence',
       'substrates',
       ...
       'socn_normalized', 'qso_combined', 'qso_normalized', 'traid_combined',
       'traid_normalized', 'paac_combined', 'paac_normalized', 'ESM1b_norm',
       'ESM1b_ts_norm', 'log10_kcat_norm'],
      dtype='object', length=2302)

**Saving Xgboost Models**

In [ ]:
%ls ../../models/train_models

 difference_fp.h5  'esm1bts_drfp (1).h5'   xgboost_diff.h5            xgboost_esm1bts_drfp.h5
 drfp.h5            esm1bts_drfp.h5        xgboost_drfp.h5            xgboost_esm1b_ts.h5
 esm1b.h5           esm1bts.h5             xgboost_esm1b.h5           xgboost_structFp.h5
 esm1bts_diff.h5    structural_fp.h5       xgboost_esm1b_ts_diff.h5


In [ ]:
def save_pickel_model(model, model_name):
    xgboost_trained_models = "../../models/train_models"
    model_path = join(xgboost_trained_models, model_name)
    with open(model_path, 'wb') as model_file:
        pickle.dump(model, model_file)

## 1. Training a model with only sequence information (ESM-1b):

#### (a) Creating input matrices:

In [ ]:
train_ESM1b = np.array(list(data_train["ESM1b"]))
train_X = train_ESM1b
train_Y = np.array(list(data_train["log10_kcat"]))

test_ESM1b = np.array(list(data_test["ESM1b"]))
test_X = test_ESM1b
test_Y = np.array(list(data_test["log10_kcat"]))

In [ ]:
for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    print(i)
    print(train_X[test_index])


0
[[ 0.07964781  0.16702731  0.22767723 ... -0.10863274 -0.14281555
   0.14292285]
 [ 0.07430488  0.17219836 -0.01209073 ... -0.13870294 -0.11353551
   0.09485441]
 [-0.01462269  0.18103217 -0.00500855 ... -0.10428829 -0.05840648
   0.01361028]
 ...
 [-0.02755747  0.18008837 -0.03818262 ... -0.03201058 -0.03821661
   0.08363783]
 [ 0.16225535  0.27029133 -0.03522961 ... -0.07823634 -0.19223613
   0.00614813]
 [ 0.05414189  0.19421057 -0.05987677 ... -0.06891491 -0.16208781
   0.05734136]]
1
[[ 0.04166316  0.181561    0.07668334 ...  0.19656172 -0.13912867
   0.03144771]
 [ 0.1302101   0.24495757 -0.02362511 ... -0.00437272 -0.07560533
   0.03059028]
 [-0.06354112 -0.01563576 -0.06732959 ... -0.20669037 -0.00257164
  -0.03227103]
 ...
 [-0.00988316  0.13642907  0.08164488 ... -0.08296695  0.02208484
   0.00863433]
 [ 0.03321628  0.13176644 -0.01363077 ... -0.12953989 -0.02000922
   0.03607321]
 [ 0.04347761  0.12669434  0.01724108 ... -0.04599944 -0.04682548
   0.17133021]]
2
[[ 0.09489

#### (b) Hyperparameter optimization:

In [ ]:
def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        print(train_index)
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)

[1, 6, 2576, 11, 2581, 2584, 16, 21, 2590, 26, 2596, 31, 2602, 2603, 36, 41, 46, 2614, 51, 2621, 2625, 57, 2626, 2628, 62, 2632, 2633, 2631, 67, 2636, 2638, 72, 2641, 77, 2647, 82, 87, 2658, 2661, 92, 2663, 97, 102, 2676, 2674, 107, 2679, 112, 2684, 117, 2689, 2693, 122, 2698, 2697, 127, 2703, 2702, 132, 2706, 2709, 137, 142, 2715, 2719, 147, 152, 2726, 2731, 157, 161, 162, 2736, 167, 2742, 2747, 172, 2748, 177, 2758, 187, 2764, 192, 2768, 197, 2773, 2778, 202, 207, 2782, 211, 216, 2792, 2794, 221, 2797, 224, 227, 2803, 229, 232, 2808, 2813, 2816, 242, 2819, 2823, 247, 2824, 249, 252, 2833, 257, 2834, 261, 2837, 263, 266, 268, 2844, 273, 274, 2848, 277, 2852, 282, 2859, 2857, 2863, 2864, 292, 2868, 297, 2873, 303, 2878, 308, 2884, 313, 2889, 2892, 2896, 2897, 323, 2901, 2909, 332, 2912, 337, 2924, 346, 2931, 2932, 356, 2939, 361, 362, 2944, 367, 2951, 2952, 373, 2953, 378, 379, 2963, 2968, 389, 2973, 2974, 399, 2978, 406, 411, 2992, 418, 2994, 2998, 422, 2999, 428, 3003, 433, 3011, 439

ERROR:hyperopt.fmin:job exception: [02:37:15] /workspace/src/tree/updater_gpu_hist.cu:781: Exception in gpu_hist: [02:37:15] /workspace/src/tree/updater_gpu_hist.cu:787: Check failed: ctx_->gpu_id >= 0 (-1 vs. 0) : Must have at least one device
Stack trace:
  [bt] (0) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0xb27f2a) [0x7ea6fdc26f2a]
  [bt] (1) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0xb3e95a) [0x7ea6fdc3d95a]
  [bt] (2) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0xb483cd) [0x7ea6fdc473cd]
  [bt] (3) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x460c79) [0x7ea6fd55fc79]
  [bt] (4) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x46176c) [0x7ea6fd56076c]
  [bt] (5) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x4c54f7) [0x7ea6fd5c44f7]
  [bt] (6) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x70) [0x7ea

  0%|          | 0/200 [00:00<?, ?trial/s, best loss=?]


XGBoostError: [02:37:15] /workspace/src/tree/updater_gpu_hist.cu:781: Exception in gpu_hist: [02:37:15] /workspace/src/tree/updater_gpu_hist.cu:787: Check failed: ctx_->gpu_id >= 0 (-1 vs. 0) : Must have at least one device
Stack trace:
  [bt] (0) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0xb27f2a) [0x7ea6fdc26f2a]
  [bt] (1) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0xb3e95a) [0x7ea6fdc3d95a]
  [bt] (2) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0xb483cd) [0x7ea6fdc473cd]
  [bt] (3) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x460c79) [0x7ea6fd55fc79]
  [bt] (4) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x46176c) [0x7ea6fd56076c]
  [bt] (5) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(+0x4c54f7) [0x7ea6fd5c44f7]
  [bt] (6) /usr/local/lib/python3.10/dist-packages/xgboost/lib/libxgboost.so(XGBoosterUpdateOneIter+0x70) [0x7ea6fd260ef0]
  [bt] (7)

In [ ]:

for train_indice, test_indice in zip(train_indices, test_indices):
    print(len(train_indice), len(test_indice))
    print(type(train_indice), type(test_indice))
    print(train_indice)

2688 703
<class 'list'> <class 'list'>
[1, 6, 2576, 11, 2581, 2584, 16, 21, 2590, 26, 2596, 31, 2602, 2603, 36, 41, 46, 2614, 51, 2621, 2625, 57, 2626, 2628, 62, 2632, 2633, 2631, 67, 2636, 2638, 72, 2641, 77, 2647, 82, 87, 2658, 2661, 92, 2663, 97, 102, 2676, 2674, 107, 2679, 112, 2684, 117, 2689, 2693, 122, 2698, 2697, 127, 2703, 2702, 132, 2706, 2709, 137, 142, 2715, 2719, 147, 152, 2726, 2731, 157, 161, 162, 2736, 167, 2742, 2747, 172, 2748, 177, 2758, 187, 2764, 192, 2768, 197, 2773, 2778, 202, 207, 2782, 211, 216, 2792, 2794, 221, 2797, 224, 227, 2803, 229, 232, 2808, 2813, 2816, 242, 2819, 2823, 247, 2824, 249, 252, 2833, 257, 2834, 261, 2837, 263, 266, 268, 2844, 273, 274, 2848, 277, 2852, 282, 2859, 2857, 2863, 2864, 292, 2868, 297, 2873, 303, 2878, 308, 2884, 313, 2889, 2892, 2896, 2897, 323, 2901, 2909, 332, 2912, 337, 2924, 346, 2931, 2932, 356, 2939, 361, 362, 2944, 367, 2951, 2952, 373, 2953, 378, 379, 2963, 2968, 389, 2973, 2974, 399, 2978, 406, 411, 2992, 418, 2994, 299

#### (c) Training and validating model:

In [ ]:
param = {'learning_rate': 0.051447544749765035,
         'max_delta_step': 2.956459783615207,
         'max_depth': 5.034202474908222,
         'min_child_weight': 7.457989829577018,
         'num_rounds': 400.50601395689256,
         'reg_alpha': 1.0858835704466614,
         'reg_lambda': 1.1385559144302175}





num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
'''
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b.npy"), np.array(Pearson))
np.save(join("..", "..", "data",  "training_results", "MSE_CV_xgboost_ESM1b.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b.npy"), np.array(R2))'''


'\nR2 = []\nMSE = []\nPearson = []\n\nfor i in range(5):\n    train_index, test_index  = train_indices[i], test_indices[i]\n    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])\n    dvalid = xgb.DMatrix(train_X[test_index])\n\n    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)\n\n    y_valid_pred = bst.predict(dvalid)\n    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))\n    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))\n    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])\n\nprint(Pearson)\nprint(MSE)\nprint(R2)\n\nnp.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b.npy"), np.array(Pearson))\nnp.save(join("..", "..", "data",  "training_results", "MSE_CV_xgboost_ESM1b.npy"), np.array(MSE))\nnp.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b.npy"), np.array(R2))'

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)

# MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
MSE_dif_fp_test = mean_squared_error(np.reshape(test_Y, (-1)), y_test_pred)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))

np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b.npy"), test_Y)

0.621 0.823 0.384


### Pearson 0.621 MSE 0.823 R2 0.384


In [ ]:
save_pickel_model(bst, "xgboost_esm1b.h5")

In [ ]:
y_test_pred_esm1b = y_test_pred

## 2. Training a model with only sequence information (ESM-1b_ts):

#### (a) Creating input matrices:

In [ ]:
train_ESM1b = np.array(list(data_train["ESM1b_ts"]))
train_X = train_ESM1b
train_Y = np.array(list(data_train["log10_kcat"]))

test_ESM1b = np.array(list(data_test["ESM1b_ts"]))
test_X = test_ESM1b
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [ ]:

'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [ ]:
param = {'learning_rate': 0.2831145406836757,
         'max_delta_step': 0.07686715986169101,
         'max_depth': 4.96836783761305,
          'min_child_weight': 6.905400087083855,
           'num_rounds': 313.1498988074061,
            'reg_alpha': 1.717314107718892,
             'reg_lambda': 2.470354543039016}

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
'''
R2 = []
MSE = []
Pearson = []
y_valid_pred_esm1b_ts = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    y_valid_pred_esm1b_ts.append(y_valid_pred)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts.npy"), np.array(Pearson))
np.save(join("..", "..", "data",  "training_results", "MSE_CV_xgboost_ESM1b_ts.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts.npy"), np.array(R2))
'''

'\nR2 = []\nMSE = []\nPearson = []\ny_valid_pred_esm1b_ts = []\n\nfor i in range(5):\n    train_index, test_index  = train_indices[i], test_indices[i]\n    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])\n    dvalid = xgb.DMatrix(train_X[test_index])\n\n    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)\n\n    y_valid_pred = bst.predict(dvalid)\n    y_valid_pred_esm1b_ts.append(y_valid_pred)\n    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))\n    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))\n    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])\n\nprint(Pearson)\nprint(MSE)\nprint(R2)\n\nnp.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts.npy"), np.array(Pearson))\nnp.save(join("..", "..", "data",  "training_results", "MSE_CV_xgboost_ESM1b_ts.npy"), np.array(MSE))\nnp.save(join("..", "..", "data", "training_resul

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))

np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b_ts.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b_ts.npy"), test_Y)

0.629 0.808 0.396


In [ ]:
save_pickel_model(bst, "xgboost_esm1b_ts.h5")

In [ ]:
y_test_pred_esm1b_ts = y_test_pred

## 3. Training a model with only reaction information (DRFP):

In [ ]:
train_X = np.array(list(data_train["DRFP"]))
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["DRFP"]))
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [ ]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [ ]:
param = {'learning_rate': 0.08987247189322463,
         'max_delta_step': 1.1939737318908727,
         'max_depth': 11.268531225242574,
         'min_child_weight': 2.8172720953826302,
         'num_rounds': 400.03643430746544,
         'reg_alpha': 1.9412226989868904,
         'reg_lambda': 4.950543905603358}


num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
'''
R2 = []
MSE = []
Pearson = []
y_valid_pred_DRFP = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    y_valid_pred_DRFP.append(y_valid_pred)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_DRFP.npy"), np.array(Pearson))
np.save(join("..", "..", "data",  "training_results", "MSE_CV_xgboost_DRFP.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_DRFP.npy"), np.array(R2))
'''

'\nR2 = []\nMSE = []\nPearson = []\ny_valid_pred_DRFP = []\n\nfor i in range(5):\n    train_index, test_index  = train_indices[i], test_indices[i]\n    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])\n    dvalid = xgb.DMatrix(train_X[test_index])\n\n    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)\n\n    y_valid_pred = bst.predict(dvalid)\n    y_valid_pred_DRFP.append(y_valid_pred)\n    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))\n    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))\n    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])\n\nprint(Pearson)\nprint(MSE)\nprint(R2)\n\nnp.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_DRFP.npy"), np.array(Pearson))\nnp.save(join("..", "..", "data",  "training_results", "MSE_CV_xgboost_DRFP.npy"), np.array(MSE))\nnp.save(join("..", "..", "data", "training_results", "R2_CV_xgbo

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))

np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_DRFP.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_DRFP.npy"), test_Y)

0.606 0.867 0.352


In [ ]:
save_pickel_model(bst, "xgboost_drfp.h5")

In [ ]:
y_test_pred_drfp = y_test_pred

## 4. Training a model with only reaction information (difference fingerprint):

#### (a) Creating input matrices:

In [ ]:
train_X = np.array(list(data_train["difference_fp"]))
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["difference_fp"]))
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [ ]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [ ]:
param = {'learning_rate': 0.14154883958006167,
         'max_delta_step': 0.02234358170535966,
         'max_depth': 10.869653004093198,
         'min_child_weight': 1.7936882442746056,
         'num_rounds': 361.6168542774665,
         'reg_alpha': 4.825525325323308,
         'reg_lambda': 2.74944090578774}


num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
'''
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_diff_fp.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_diff_fp.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_diff_fp.npy"), np.array(R2))
'''

'\nR2 = []\nMSE = []\nPearson = []\n\nfor i in range(5):\n    train_index, test_index  = train_indices[i], test_indices[i]\n    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])\n    dvalid = xgb.DMatrix(train_X[test_index])\n\n    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)\n\n    y_valid_pred = bst.predict(dvalid)\n    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))\n    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))\n    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])\n\nprint(Pearson)\nprint(MSE)\nprint(R2)\n\nnp.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_diff_fp.npy"), np.array(Pearson))\nnp.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_diff_fp.npy"), np.array(MSE))\nnp.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_diff_fp.npy"), np.array(R2))\n'

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_diff_fp.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_diff_fp.npy"), test_Y)

0.624 0.849 0.365


In [ ]:
save_pickel_model(bst, "xgboost_diff.h5")


In [ ]:
y_test_pred_diff_fp = y_test_pred

## 5. Training a model with only reaction information (structural fingerprint):

#### (a) Creating input matrices:

In [ ]:
train_X = np.array(data_train["structural_fp"].tolist())
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["structural_fp"]))
test_Y = np.array(list(data_test["log10_kcat"]))

In [ ]:
train_X.shape, test_X.shape

((3391, 4096), (874, 4096))

#### (b) Hyperparameter optimization:

In [ ]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [ ]:
param = {'learning_rate': 0.01126910440903659,
         'max_delta_step': 0.5777120839605732,
         'max_depth': 5.486901609313889,
         'min_child_weight': 6.14467742389769,
         'num_rounds': 488.943459090126,
         'reg_alpha': 4.629840853377147,
         'reg_lambda': 2.1047561335691745}

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
'''
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_str_fp.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_str_fp.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_str_fp.npy"), np.array(R2))
'''

'\nR2 = []\nMSE = []\nPearson = []\n\nfor i in range(5):\n    train_index, test_index  = train_indices[i], test_indices[i]\n    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])\n    dvalid = xgb.DMatrix(train_X[test_index])\n\n    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)\n\n    y_valid_pred = bst.predict(dvalid)\n    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))\n    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))\n    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])\n\nprint(Pearson)\nprint(MSE)\nprint(R2)\n\nnp.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_str_fp.npy"), np.array(Pearson))\nnp.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_str_fp.npy"), np.array(MSE))\nnp.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_str_fp.npy"), np.array(R2))\n'

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_str_fp.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_str_fp.npy"), test_Y)

0.567 0.918 0.313


In [ ]:
save_pickel_model(bst, "xgboost_structFp.h5")

## 6. Training a model with enzyme and reaction information (ESM1b_ts/DRFP):

In [ ]:
train_X = np.array(list(data_train["DRFP"]))
train_X = np.concatenate([train_X, np.array(list(data_train["ESM1b_ts"]))], axis = 1)
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["DRFP"]))
test_X = np.concatenate([test_X, np.array(list(data_test["ESM1b_ts"]))], axis = 1)
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [ ]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 6,14),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [ ]:
param =  {
    'learning_rate': 0.0685249638804391,
    'max_depth': 9,
    'reg_lambda': 3.6231955480293934,
    'reg_alpha': 3.8923304126950957,
    'max_delta_step': 1.0371427260254895,
    'min_child_weight': 14.209819958496073,
    # 'sampling_method': 'gradient_based',
    'num_rounds': 400.9598325756988,
    }


num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
'''
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts_DRFP.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_ESM1b_ts_DRFP.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts_DRFP.npy"), np.array(R2))
'''

'\nR2 = []\nMSE = []\nPearson = []\n\nfor i in range(5):\n    train_index, test_index  = train_indices[i], test_indices[i]\n    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])\n    dvalid = xgb.DMatrix(train_X[test_index])\n\n    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)\n\n    y_valid_pred = bst.predict(dvalid)\n    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))\n    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))\n    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])\n\nprint(Pearson)\nprint(MSE)\nprint(R2)\n\nnp.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts_DRFP.npy"), np.array(Pearson))\nnp.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_ESM1b_ts_DRFP.npy"), np.array(MSE))\nnp.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts_DRFP.npy"), np.array(R2))\n'

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X, label = test_Y)


bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
# MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
MSE_dif_fp_test = mean_squared_error(np.reshape(test_Y, (-1)), y_test_pred)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b_ts_DRFP.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b_ts_DRFP.npy"), test_Y)
save_pickel_model(bst, "xgboost_esm1bts_drfp.h5")

0.656 0.766 0.427


### Pearson = 0.656 MSE = 0.766 R2 = 0.427

In [ ]:
y_test_pred_esm1b_ts_drfp = y_test_pred

## 7. Training a model with enzyme and reaction information (ESM1b_ts/diff_fp):

#### (a) Creating input matrices:

In [ ]:
train_X = np.array(list(data_train["difference_fp"]))
train_X = np.concatenate([train_X, np.array(list(data_train["ESM1b_ts"]))], axis = 1)
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["difference_fp"]))
test_X = np.concatenate([test_X, np.array(list(data_test["ESM1b_ts"]))], axis = 1)
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [ ]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [ ]:
param = {
    'learning_rate': 0.15638876027725748,
    'max_depth': 8,
    'reg_lambda': 2.344616112944763,
    'reg_alpha': 4.262441473730964,
    'max_delta_step': 0.9349225200130806,
    'min_child_weight': 8.26447488212503,
    'num_rounds': 400.69265795096726,
    # 'sampling_method': 'gradient_based'
    }

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
'''
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(R2))
'''

'\nR2 = []\nMSE = []\nPearson = []\n\nfor i in range(5):\n    train_index, test_index  = train_indices[i], test_indices[i]\n    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])\n    dvalid = xgb.DMatrix(train_X[test_index])\n\n    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)\n\n    y_valid_pred = bst.predict(dvalid)\n    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))\n    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))\n    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])\n\nprint(Pearson)\nprint(MSE)\nprint(R2)\n\nnp.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(Pearson))\nnp.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(MSE))\nnp.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(R2))\n'

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X, label = test_Y)


bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b_ts_diff_fp.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b_ts_diff_fp.npy"), test_Y)

save_pickel_model(bst, "xgboost_esm1b_ts_diff.h5")

0.642 0.787 0.412


In [ ]:
y_test_pred_esm1b_ts_drfp = y_test_pred

## Training model using all features

In [ ]:
train_indices = list(np.load(join("..", "..", "data", "kcat_data", "splits", "CV_train_indices.npy"), allow_pickle = True))
test_indices = list(np.load(join("..", "..", "data", "kcat_data", "splits", "CV_test_indices.npy"), allow_pickle = True))

In [ ]:
data_train = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_ours.pkl"))
data_test = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_ours.pkl"))

# print((train_indices))
# print((test_indices))

data_train.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
data_test.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
len(data_train), len(data_test)

(3391, 874)

In [ ]:
data_train.columns[:-7]

Index(['Reaction ID', 'Sequence ID', 'kcat_values', 'Uniprot IDs',
       'from_BRENDA', 'from_Sabio', 'from_Uniprot', 'checked', 'Sequence',
       'substrates',
       ...
       '765', '766', '767', '771', '772', '773', '774', '775', '776', '777'],
      dtype='object', length=2270)

In [ ]:
np.array(data_train['structural_fp'].tolist())[0]

array([1, 0, 0, ..., 0, 0, 0])

#### (a) Creating input matrices:

In [ ]:
train_X = np.array(list(data_train["ESM1b"]))
print(train_X.shape)
test_X = np.array(list(data_test["ESM1b"]))
for col in data_train.columns[17:21]:
  print(col)
  if col == 'structural_fp':
    train_X = np.concatenate([train_X, np.array(data_train[col].tolist(), dtype='float64')], axis = 1)
    test_X = np.concatenate([test_X, np.array(data_test[col].tolist(), dtype='float64')], axis = 1)
  else:
    train_X = np.concatenate([train_X, np.array(list(data_train[col]), dtype='float64')], axis = 1)
    test_X = np.concatenate([test_X, np.array(list(data_test[col]), dtype='float64')], axis = 1)

(3391, 1280)
ESM1b_ts
structural_fp
difference_fp
DRFP


In [ ]:
train_X = np.concatenate([train_X, np.array(list(data_train['ESM1b_norm']), dtype='float64')], axis = 1)
test_X = np.concatenate([test_X, np.array(list(data_test['ESM1b_norm']), dtype='float64')], axis = 1)
train_X = np.concatenate([train_X, np.array(list(data_train['ESM1b_ts_norm']), dtype='float64')], axis = 1)
test_X = np.concatenate([test_X, np.array(list(data_test['ESM1b_ts_norm']), dtype='float64')], axis = 1)

In [ ]:
train_X = np.concatenate([train_X, np.array(data_train.iloc[:,22:-7], dtype='float64')], axis = 1)
test_X = np.concatenate([test_X, np.array(data_test.iloc[:,22:-7], dtype='float64')], axis = 1)

In [ ]:
train_X = train_X.astype('float64')
test_X = test_X.astype('float64')

In [ ]:
train_Y = np.array(list(data_train["log10_kcat"]))
test_Y = np.array(list(data_test["log10_kcat"]))

In [ ]:
train_X.shape

(3391, 15560)

In [ ]:
type(train_X)

numpy.ndarray

#### (b) Hyperparameter optimization:

In [ ]:
def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)

 87%|████████▋ | 174/200 [5:08:10<40:19, 93.06s/trial, best loss: -0.3735840090139679]

In [ ]:
best

#### (c) Training and validating model:

In [ ]:
param = {
    'learning_rate': 0.15638876027725748,
    'max_depth': 8,
    'reg_lambda': 2.344616112944763,
    'reg_alpha': 4.262441473730964,
    'max_delta_step': 0.9349225200130806,
    'min_child_weight': 8.26447488212503,
    'num_rounds': 400.69265795096726,
    # 'sampling_method': 'gradient_based'
    }

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:

R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_all.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_all.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_all.npy"), np.array(R2))


[0.6133773796060704, 0.5927733649751115, 0.5696579476183089, 0.5664901288879961, 0.6059675908688378]
[0.8995350132420895, 0.8907148311609805, 0.9570658132750683, 1.0011614220188685, 0.9055139379678919]
[0.37289620835138915, 0.3481887641919241, 0.3220370010024399, 0.31963431757814464, 0.3656110621769998]


In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X, label = test_Y)


bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_all.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_all.npy"), test_Y)

save_pickel_model(bst, "xgboost_all.h5")

0.613 0.835 0.375


In [ ]:
y_test_pred_esm1b_ts_drfp = y_test_pred